<a href="https://colab.research.google.com/github/ppgiii/Data-Visualization/blob/master/eetest2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [20]:
!earthengine authenticate

Instructions for updating:
non-resource variables are not supported in the long term
Running command using Cloud API.  Set --no-use_cloud_api to go back to using the API

To authorize access needed by Earth Engine, open the following URL in a web browser and follow the instructions. If the web browser does not start automatically, please manually browse the URL below.

    https://accounts.google.com/o/oauth2/auth?client_id=517222506229-vsmmajv00ul0bs7p89v5m89qs8eb9359.apps.googleusercontent.com&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fearthengine+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdevstorage.full_control&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&response_type=code&code_challenge=El-tS4YDVX-wvTdDBsgHObZTBFB2KHOmWVL-HREW47U&code_challenge_method=S256

The authorization workflow will generate a code, which you should paste in the box below. 
Enter verification code: 4/3AEdgzCSwlt6Zt61vRZTG_kpn273LWAz4SKZD5a6Y7-12QZuZkax0BM

Successfully saved authorization token.


In [21]:
import ee
# Trigger the authentication flow.
ee.Authenticate()

# Initialize the library.
ee.Initialize()

To authorize access needed by Earth Engine, open the following URL in a web browser and follow the instructions. If the web browser does not start automatically, please manually browse the URL below.

    https://accounts.google.com/o/oauth2/auth?client_id=517222506229-vsmmajv00ul0bs7p89v5m89qs8eb9359.apps.googleusercontent.com&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fearthengine+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdevstorage.full_control&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&response_type=code&code_challenge=buqE-HC3fV2HYqQnzS1WEHI9_uGcQvckZ2MOUoGFU1U&code_challenge_method=S256

The authorization workflow will generate a code, which you should paste in the box below. 
Enter verification code: 4/3AG9LDwF8xCOMIvs0GUKvMnOWkufMQmZGi2cePgeckktjNVh6pn5u4I

Successfully saved authorization token.


In [22]:
col = ee.ImageCollection('MODIS/006/MOD13A2').select('NDVI')
mask = ee.FeatureCollection('USDOS/LSIB_SIMPLE/2017').filter(ee.Filter.eq('country_co', 'IN'))

In [23]:
region = ee.Geometry.Polygon( \
  [[\
    [74.86723005960881,22.403438033015952],\
    [81.22831404398381,22.403438033015952],\
    [81.22831404398381,25.435860310982335],\
    [74.86723005960881,25.435860310982335],\
    [74.86723005960881,22.403438033015952]\
  ]],  None, False)

In [24]:
col = col.map(lambda img: img.set('doy', ee.Date(img.get('system:time_start')).getRelative('day', 'year') ))

In [54]:
distinctDOY = col.filterDate('2013-01-01', '2019-01-01');
filter = ee.Filter.equals('doy', None, 'doy', None)

In [55]:
join = ee.Join.saveAll('doy_matches')

In [56]:
joinCol = ee.ImageCollection(join.apply(distinctDOY, col, filter))

In [57]:
comp = joinCol.map(lambda img: ee.ImageCollection.fromImages(img.get('doy_matches')).reduce(ee.Reducer.median()) )

In [140]:
palette = [ \
    'FFFFFF', 'CE7E45', 'DF923D', 'F1B555', 'FCD163', '99B718', '74A901',\
    '66A000', '529400', '3E8601', '207401', '056201', '004C00', '023B01',\
    '012E01', '011D01', '011301'\
]

In [141]:
rgbVis = comp.map(lambda img: img.visualize(None, None, None, 0.0, 9000.0, None, None, palette).clip(mask) )

In [142]:
gifParams = {\
  'region': region,\
  'dimensions': 600,\
  'crs': 'EPSG:3857',\
  'framesPerSecond': 10\
}

In [143]:
print(rgbVis.getVideoThumbURL(gifParams))

https://earthengine.googleapis.com/v1alpha/projects/earthengine-legacy/videoThumbnails/63355ed56074d63ce80852338e26d7b2-418e008ea06d80d15e26df26de5ee7b7:getPixels
